# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Как будет происходить сдача ДЗ

Вам надо:
- форкнуть эту репу
- создать бранч в котором вы дальше будете работать
- реализовать класс Model в этом ноутбуке
- убедиться, что ваша реализация выбивает NDCG@10 выше бейзлайна (см. ниже)
- запушить ваш бранч и поставить Pull Request
- в комментарии написать какой скор вы выбили

В таком случае мы (организаторы):

- счекаутим вашу бранчу
- проверим что ваша реализация действительно выбивает заявленный скор

Предполагается, что и вы, и мы работаем в виртаульном окружении как в семинаре про машинное обучение ранжированию: seminars/7-learning-to-rank/requirements.txt(подробнее про работу с виртуальными окружениями README в корне этой репы).

Оценка:
- За выбитый скор больше **0.507** назначаем **5** баллов, за скор больше (или равно) **0.510** назначаем максимальный балл -- 10 баллов
- Тот из участников кто выбъет самый высокий скор получит еще +10 баллов

При сдаче кода важно помнить о том, что:
- В коде не должно быть захардкоженных с потолка взятых гиперпараметров (таких как число деревьев, learning rate и т.п.) -- обязательно должен быть представлен код который их подбирает!
- Решение должно быть стабильно от запуска к запуску (на CPU) т.е. все seed'ы для генераторов случайных чисел должны быть фиксированы
- Мы (организаторы) будем запускать код на CPU поэтому, даже если вы использовали для подбора параметров GPU, финальный скор надо репортить на CPU

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [ ]:
!pip install catboost

In [2]:
import pathlib
from timeit import default_timer as timer

import numpy as np
import pandas as pd

from catboost import datasets, utils

In [3]:
import copy
import catboost

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет MSLR-WEB10K с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

In [ ]:
!mkdir data

!wget https://storage.googleapis.com/personalization-takehome/MSLR-WEB10K.zip

!unzip MSLR-WEB10K.zip -d data

!rm MSLR-WEB10K.zip

In [5]:
!ls -lh data

total 20K
drwxr-xr-x 2 root root 4.0K Apr 28  2010 Fold1
drwxr-xr-x 2 root root 4.0K Apr 28  2010 Fold2
drwxr-xr-x 2 root root 4.0K Apr 28  2010 Fold3
drwxr-xr-x 2 root root 4.0K Apr 28  2010 Fold4
drwxr-xr-x 2 root root 4.0K Apr 28  2010 Fold5


Заметим, что датасет довольно большой, в распакованном виде он весит 7.7 GB.

Датасет состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [6]:
!ls -lh data/Fold1

total 1.3G
-rw-r--r-- 1 root root 267M Apr 30  2010 test.txt
-rw-r--r-- 1 root root 800M Apr 30  2010 train.txt
-rw-r--r-- 1 root root 261M Apr 30  2010 vali.txt


Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [7]:
!head -n 1 data/Fold1/train.txt

2 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 16:6.931275 17:22.076928 18:19.673353 19:22.255383 20:6.926551 21:3 22:3 23:0 24:0 25:6 26:1 27:1 28:0 29:0 30:2 31:1 32:1 33:0 34:0 35:2 36:1 37:1 38:0 39:0 40:2 41:0 42:0 43:0 44:0 45:0 46:0.019231 47:0.75000 48:0 49:0 50:0.035928 51:0.00641 52:0.25000 53:0 54:0 55:0.011976 56:0.00641 57:0.25000 58:0 59:0 60:0.011976 61:0.00641 62:0.25000 63:0 64:0 65:0.011976 66:0 67:0 68:0 69:0 70:0 71:6.931275 72:22.076928 73:0 74:0 75:13.853103 76:1.152128 77:5.99246 78:0 79:0 80:2.297197 81:3.078917 82:8.517343 83:0 84:0 85:6.156595 86:2.310425 87:7.358976 88:0 89:0 90:4.617701 91:0.694726 92:1.084169 93:0 94:0 95:2.78795 96:1 97:1 98:0 99:0 100:1 101:1 102:1 103:0 104:0 105:1 106:12.941469 107:20.59276 108:0 109:0 110:16.766961 111:-18.567793 112:-7.760072 113:-20.838749 114:-25.436074 115:-14.518523 116:-21.710022 117:-21.339609 118:-24.497864 119:-27.690319 120:-20.203779 121:-15.449379 122:-4.474452 123:-23.634899 

Видим, что данные лежат в уже знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [8]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns

def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1

                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")

                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)

                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e

    # To pandas
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [9]:
fold_dir = pathlib.Path("data/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Loaded SVM-Light file data/Fold1/train.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file data/Fold1/vali.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file data/Fold1/test.txt
Dataset loaded from fold_dir data/Fold1


Посмотрим на данные:

In [10]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0  1.0        3.0        3.0        0.0        0.0        3.0   
1    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
2    0.0  1.0        3.0        0.0        2.0        0.0        3.0   
3    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
4    1.0  1.0        3.0        0.0        3.0        0.0        3.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...         62.0   11089534.0   
1        1.0        0.0   1.000000  ...         54.0   11089534.0   
2        1.0        0.0   0.666667  ...         45.0          3.0   
3        1.0        0.0   1.000000  ...         56.0   11089534.0   
4        1.0        0.0   1.000000  ...         64.0          5.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          2.0        116.0      64034.0         13.0          3.0   
1          2

Т.е. теперь мы видим что данные доступны в точно таком же виде, как это было в семинаре.

Проведем небольшой EDA.

Всего у нас 723 тыс. документов в трейне:

In [11]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723412 entries, 0 to 723411
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 380.8 MB
None


235 тыс. документов в валидации:

In [12]:
print(df_valid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235259 entries, 0 to 235258
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 123.8 MB
None


И 241 тыс. документов в тесте:

In [13]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241521 entries, 0 to 241520
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 127.1 MB
None


Сколько у нас запросов?

In [14]:
num_queries_train = df_train['qid'].nunique()
num_queries_valid = df_valid['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train, {num_queries_valid} valid and {num_queries_test} test queries")

Got 6000 train, 2000 valid and 2000 test queries


## Обучаем модель

Теперь можно приступить непосредственно к обучению модели.

Объявим класс модели, который надо будем заимлементить в этом ДЗ:

In [15]:
def to_catboost_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy().astype('uint32')        # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
    return (X, y, q)

In [16]:
X_train, y_train, q_train = to_catboost_dataset(df_train)
X_valid, y_valid, q_valid = to_catboost_dataset(df_valid)
X_test, y_test, q_test = to_catboost_dataset(df_test)

In [17]:
pool_train = catboost.Pool(data=X_train, label=y_train, group_id=q_train)
pool_valid = catboost.Pool(data=X_valid, label=y_valid, group_id=q_valid)
pool_test = catboost.Pool(data=X_test, label=y_test, group_id=q_test)

In [18]:
class Model:
    def __init__(self, params):
        self.model = catboost.CatBoost(params)

    def fit(self, df_train, df_valid):
        self.model.fit(df_train, eval_set=df_valid, use_best_model=True)

    def predict(self, df_test):
        return self.model.predict(df_test)

    def save(self, filename):
        self.model.save_model(filename)

    def load(self, filename):
        self.model = catboost.CatBoost()
        self.model.load_model(filename)

In [19]:
def get_best_model(pool_train, pool_valid):
    best_score = 0
    best_model = None
    best_params = {}

    y_valid = pool_valid.get_label()
    q_valid = pool_valid.get_group_id_hash()

    for lr in [0.001, 0.005, 0.01, 0.05, 0.1]:
        for depth in [3, 5, 7, 9]:
            for l2_leaf_reg in [0.5, 1, 1.5, 2, 3]:

                params = {
                    'iterations': 1500,
                    'early_stopping_rounds': 200,
                    'eval_metric': "NDCG:top=10;type=Exp",
                    'random_seed': 22,
                    'verbose': 100,
                    'loss_function': 'YetiRank',
                    'train_dir': "/tmp/catboost_info.YetiRank",
                    'use_best_model': True,
                    'task_type': "GPU",
                    'learning_rate': lr,
                    'depth': depth,
                    'l2_leaf_reg': l2_leaf_reg,
                }

                model = Model(params)
                model.fit(pool_train, pool_valid)
                y_pred = model.predict(pool_valid)
                score = utils.eval_metric(y_valid, y_pred, 'NDCG:top=10;type=Exp', group_id=q_valid)[0]

                if score > best_score:
                    best_model = model
                    best_score = score
                    best_params = copy.deepcopy(params)

    return best_model, best_params

In [ ]:
get_best_model(pool_train, pool_valid)

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2341477	best: 0.2341477 (0)	total: 89.5ms	remaining: 2m 14s
100:	test: 0.3310440	best: 0.3310440 (100)	total: 3.9s	remaining: 54.1s
200:	test: 0.3418268	best: 0.3418268 (200)	total: 8.55s	remaining: 55.3s
300:	test: 0.3479658	best: 0.3482059 (289)	total: 12.6s	remaining: 50.2s
400:	test: 0.3627104	best: 0.3627104 (400)	total: 16.4s	remaining: 44.9s
500:	test: 0.3636218	best: 0.3636218 (500)	total: 21.2s	remaining: 42.3s
600:	test: 0.3646954	best: 0.3648218 (575)	total: 25s	remaining: 37.3s
700:	test: 0.3692506	best: 0.3693486 (681)	total: 28.7s	remaining: 32.7s
800:	test: 0.3712281	best: 0.3712869 (799)	total: 33.4s	remaining: 29.1s
900:	test: 0.3734938	best: 0.3735306 (895)	total: 37.2s	remaining: 24.7s
1000:	test: 0.3744694	best: 0.3746072 (996)	total: 40.9s	remaining: 20.4s
1100:	test: 0.3754285	best: 0.3754309 (1098)	total: 45.6s	remaining: 16.5s
1200:	test: 0.3765116	best: 0.3765393 (1195)	total: 49.4s	remaining: 12.3s
1300:	test: 0.3782441	best: 0.3783030 (1299)	total:

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2341477	best: 0.2341477 (0)	total: 156ms	remaining: 3m 53s
100:	test: 0.3310247	best: 0.3310247 (100)	total: 4.65s	remaining: 1m 4s
200:	test: 0.3417647	best: 0.3417884 (199)	total: 8.36s	remaining: 54s
300:	test: 0.3481776	best: 0.3482844 (289)	total: 12.4s	remaining: 49.4s
400:	test: 0.3630253	best: 0.3630253 (400)	total: 16.8s	remaining: 46.1s
500:	test: 0.3637593	best: 0.3638860 (492)	total: 20.5s	remaining: 41s
600:	test: 0.3649750	best: 0.3650237 (594)	total: 24.4s	remaining: 36.6s
700:	test: 0.3690740	best: 0.3691246 (694)	total: 29s	remaining: 33.1s
800:	test: 0.3712719	best: 0.3713650 (792)	total: 32.8s	remaining: 28.6s
900:	test: 0.3735615	best: 0.3736754 (896)	total: 36.6s	remaining: 24.3s
1000:	test: 0.3742090	best: 0.3742907 (995)	total: 41.3s	remaining: 20.6s
1100:	test: 0.3756283	best: 0.3756286 (1098)	total: 45.1s	remaining: 16.3s
1200:	test: 0.3764397	best: 0.3765336 (1192)	total: 49.3s	remaining: 12.3s
1300:	test: 0.3780086	best: 0.3781039 (1297)	total: 53.

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3307149	best: 0.3307149 (0)	total: 115ms	remaining: 2m 52s
100:	test: 0.4714996	best: 0.4714996 (100)	total: 6s	remaining: 1m 23s
200:	test: 0.4898785	best: 0.4898785 (200)	total: 11.9s	remaining: 1m 17s
300:	test: 0.4961918	best: 0.4962141 (297)	total: 17.5s	remaining: 1m 9s
400:	test: 0.4995071	best: 0.4995071 (400)	total: 23.6s	remaining: 1m 4s
500:	test: 0.5017734	best: 0.5020849 (494)	total: 29s	remaining: 57.9s
600:	test: 0.5046053	best: 0.5046053 (600)	total: 35.3s	remaining: 52.8s
700:	test: 0.5058794	best: 0.5060304 (696)	total: 40.6s	remaining: 46.3s
800:	test: 0.5077836	best: 0.5079717 (793)	total: 46.9s	remaining: 40.9s
900:	test: 0.5087197	best: 0.5089896 (895)	total: 52.2s	remaining: 34.7s
1000:	test: 0.5096210	best: 0.5102371 (972)	total: 58.3s	remaining: 29.1s
1100:	test: 0.5108154	best: 0.5108154 (1100)	total: 1m 3s	remaining: 23.1s
1200:	test: 0.5107358	best: 0.5109048 (1196)	total: 1m 9s	remaining: 17.4s
1300:	test: 0.5114162	best: 0.5116372 (1294)	total: 

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3294239	best: 0.3294239 (0)	total: 115ms	remaining: 2m 52s
100:	test: 0.4699529	best: 0.4699529 (100)	total: 5.72s	remaining: 1m 19s
200:	test: 0.4885940	best: 0.4888345 (199)	total: 12s	remaining: 1m 17s
300:	test: 0.4952654	best: 0.4952654 (300)	total: 17.4s	remaining: 1m 9s
400:	test: 0.4993533	best: 0.4993533 (400)	total: 23.6s	remaining: 1m 4s
500:	test: 0.5022992	best: 0.5023615 (498)	total: 29s	remaining: 57.8s
600:	test: 0.5040583	best: 0.5040583 (600)	total: 35.2s	remaining: 52.7s
700:	test: 0.5059602	best: 0.5059688 (699)	total: 40.6s	remaining: 46.3s
800:	test: 0.5076083	best: 0.5076083 (800)	total: 46.5s	remaining: 40.6s
900:	test: 0.5087662	best: 0.5088194 (899)	total: 52s	remaining: 34.6s
1000:	test: 0.5090124	best: 0.5092460 (998)	total: 57.6s	remaining: 28.7s
1100:	test: 0.5096528	best: 0.5098555 (1096)	total: 1m 3s	remaining: 23.1s
1200:	test: 0.5105400	best: 0.5107640 (1187)	total: 1m 9s	remaining: 17.2s
1300:	test: 0.5111346	best: 0.5115124 (1276)	total: 1

(<__main__.Model at 0x79f56b401e10>,
 {'iterations': 1500,
  'early_stopping_rounds': 200,
  'eval_metric': 'NDCG:top=10;type=Exp',
  'random_seed': 22,
  'verbose': 100,
  'loss_function': 'YetiRank',
  'train_dir': '/tmp/catboost_info.YetiRank',
  'use_best_model': True,
  'task_type': 'GPU',
  'learning_rate': 0.1,
  'depth': 9,
  'l2_leaf_reg': 0.5})

In [21]:
def get_best_model(pool_train, pool_valid):
    best_score = 0
    best_model = None
    best_params = {}

    y_valid = pool_valid.get_label()
    q_valid = pool_valid.get_group_id_hash()


    for min_data_in_leaf in [10, 50, 100]:
        for bagging_temperature in [0.5, 0.6, 0.7, 0.8, 0.9, 1]:
            params = {
              'iterations': 2000,
              'eval_metric': 'NDCG:top=10;type=Exp',
              'random_seed': 22,
              'verbose': 100,
              'loss_function': 'YetiRank',
              'train_dir': '/tmp/catboost_info.YetiRank',
              'use_best_model': True,
              'task_type': 'GPU',
              'learning_rate': 0.1,
              'depth': 9,
              'l2_leaf_reg': 0.5,
              'min_data_in_leaf': min_data_in_leaf,
              'bagging_temperature': bagging_temperature
            }

            model = Model(params)
            model.fit(pool_train, pool_valid)
            y_pred = model.predict(pool_valid)
            score = utils.eval_metric(y_valid, y_pred, 'NDCG:top=10;type=Exp', group_id=q_valid)[0]

            if score > best_score:
                best_model = model
                best_score = score
                best_params = copy.deepcopy(params)

    return best_model, best_params

In [ ]:
model, params = get_best_model(pool_train, pool_valid)
params

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3361778	best: 0.3361778 (0)	total: 184ms	remaining: 6m 7s
100:	test: 0.4760278	best: 0.4760278 (100)	total: 5.57s	remaining: 1m 44s
200:	test: 0.4931763	best: 0.4931763 (200)	total: 11.3s	remaining: 1m 40s
300:	test: 0.5003196	best: 0.5003196 (300)	total: 16.9s	remaining: 1m 35s
400:	test: 0.5026095	best: 0.5026491 (390)	total: 21.5s	remaining: 1m 25s
500:	test: 0.5056198	best: 0.5057437 (480)	total: 27s	remaining: 1m 20s
600:	test: 0.5077461	best: 0.5077461 (600)	total: 31.6s	remaining: 1m 13s
700:	test: 0.5093745	best: 0.5093745 (700)	total: 36.1s	remaining: 1m 6s
800:	test: 0.5099419	best: 0.5100270 (795)	total: 41.8s	remaining: 1m 2s
900:	test: 0.5119062	best: 0.5119062 (900)	total: 46.4s	remaining: 56.6s
1000:	test: 0.5131142	best: 0.5131142 (1000)	total: 51.9s	remaining: 51.8s
1100:	test: 0.5130908	best: 0.5135149 (1025)	total: 56.4s	remaining: 46.1s
1200:	test: 0.5138190	best: 0.5138190 (1200)	total: 1m 1s	remaining: 40.7s
1300:	test: 0.5148871	best: 0.5148871 (1300)	

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3385121	best: 0.3385121 (0)	total: 114ms	remaining: 3m 48s
100:	test: 0.4766997	best: 0.4766997 (100)	total: 5.92s	remaining: 1m 51s
200:	test: 0.4933520	best: 0.4933520 (200)	total: 10.5s	remaining: 1m 34s
300:	test: 0.4997445	best: 0.4998338 (295)	total: 16.2s	remaining: 1m 31s
400:	test: 0.5028597	best: 0.5028597 (400)	total: 20.7s	remaining: 1m 22s
500:	test: 0.5054114	best: 0.5054114 (500)	total: 25.3s	remaining: 1m 15s
600:	test: 0.5074004	best: 0.5074004 (600)	total: 30.9s	remaining: 1m 11s
700:	test: 0.5094604	best: 0.5094604 (700)	total: 35.5s	remaining: 1m 5s
800:	test: 0.5105225	best: 0.5107197 (795)	total: 41s	remaining: 1m 1s
900:	test: 0.5108885	best: 0.5110147 (885)	total: 45.6s	remaining: 55.6s
1000:	test: 0.5118033	best: 0.5120298 (990)	total: 50.1s	remaining: 50s
1100:	test: 0.5127320	best: 0.5127956 (1095)	total: 55.8s	remaining: 45.5s
1200:	test: 0.5134771	best: 0.5136274 (1175)	total: 1m	remaining: 40.1s
1300:	test: 0.5140717	best: 0.5141096 (1280)	total

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

0:	test: 0.3393252	best: 0.3393252 (0)	total: 113ms	remaining: 3m 46s
100:	test: 0.4750090	best: 0.4750090 (100)	total: 5.52s	remaining: 1m 43s
200:	test: 0.4917042	best: 0.4919477 (190)	total: 10.5s	remaining: 1m 33s
300:	test: 0.4989136	best: 0.4989136 (300)	total: 15.1s	remaining: 1m 25s
400:	test: 0.5025036	best: 0.5025036 (400)	total: 20.7s	remaining: 1m 22s
500:	test: 0.5047216	best: 0.5047216 (500)	total: 25.2s	remaining: 1m 15s
600:	test: 0.5071603	best: 0.5071638 (590)	total: 30.8s	remaining: 1m 11s
700:	test: 0.5086095	best: 0.5086095 (700)	total: 35.4s	remaining: 1m 5s
800:	test: 0.5102296	best: 0.5102807 (790)	total: 40.1s	remaining: 60s
900:	test: 0.5108251	best: 0.5108251 (900)	total: 46.6s	remaining: 56.9s
1000:	test: 0.5117805	best: 0.5121749 (985)	total: 51.2s	remaining: 51.1s
1100:	test: 0.5127641	best: 0.5128764 (1080)	total: 56.7s	remaining: 46.3s
1200:	test: 0.5133702	best: 0.5133702 (1200)	total: 1m 1s	remaining: 40.8s
1300:	test: 0.5135787	best: 0.5140809 (1240)	

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3160694	best: 0.3160694 (0)	total: 119ms	remaining: 3m 57s
100:	test: 0.4755216	best: 0.4755216 (100)	total: 4.93s	remaining: 1m 32s
200:	test: 0.4930752	best: 0.4930752 (200)	total: 10.5s	remaining: 1m 34s
300:	test: 0.4992085	best: 0.4995322 (295)	total: 15.2s	remaining: 1m 25s
400:	test: 0.5020426	best: 0.5023068 (380)	total: 20.2s	remaining: 1m 20s
500:	test: 0.5058282	best: 0.5058282 (500)	total: 25.3s	remaining: 1m 15s
600:	test: 0.5075126	best: 0.5075126 (600)	total: 29.9s	remaining: 1m 9s
700:	test: 0.5092861	best: 0.5092861 (700)	total: 35.5s	remaining: 1m 5s
800:	test: 0.5105143	best: 0.5106853 (795)	total: 40.1s	remaining: 1m
900:	test: 0.5119313	best: 0.5119313 (900)	total: 45.4s	remaining: 55.4s
1000:	test: 0.5130155	best: 0.5131515 (990)	total: 50.3s	remaining: 50.2s
1100:	test: 0.5144734	best: 0.5144759 (1090)	total: 54.9s	remaining: 44.8s
1200:	test: 0.5144098	best: 0.5145257 (1110)	total: 1m	remaining: 40.3s
1300:	test: 0.5150204	best: 0.5150204 (1300)	total

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3242144	best: 0.3242144 (0)	total: 154ms	remaining: 5m 8s


In [22]:
#best params
params = {
    'iterations': 6000,
    'eval_metric': 'NDCG:top=10;type=Exp',
    'random_seed': 22,
    'verbose': 100,
    'loss_function': 'YetiRank',
    'train_dir': '/tmp/catboost_info.YetiRank',
    'use_best_model': True,
    'task_type': 'GPU',
    'learning_rate': 0.1,
    'depth': 9,
    'l2_leaf_reg': 0.5,
    'bagging_temperature': 0.5
}

Создадим и применим модель:

In [23]:
# Create model
model = Model(params)

# Fit
start = timer()
model.fit(pool_train, pool_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3361778	best: 0.3361778 (0)	total: 402ms	remaining: 40m 13s
100:	test: 0.4760278	best: 0.4760278 (100)	total: 6.29s	remaining: 6m 7s
200:	test: 0.4931763	best: 0.4931763 (200)	total: 11.9s	remaining: 5m 44s
300:	test: 0.5003196	best: 0.5003196 (300)	total: 16.4s	remaining: 5m 10s
400:	test: 0.5026095	best: 0.5026491 (390)	total: 21s	remaining: 4m 52s
500:	test: 0.5056198	best: 0.5057437 (480)	total: 26.3s	remaining: 4m 48s
600:	test: 0.5077461	best: 0.5077461 (600)	total: 30.8s	remaining: 4m 36s
700:	test: 0.5093745	best: 0.5093745 (700)	total: 36.3s	remaining: 4m 34s
800:	test: 0.5099419	best: 0.5100270 (795)	total: 40.8s	remaining: 4m 24s
900:	test: 0.5119062	best: 0.5119062 (900)	total: 45.2s	remaining: 4m 16s
1000:	test: 0.5131142	best: 0.5131142 (1000)	total: 50.8s	remaining: 4m 13s
1100:	test: 0.5130908	best: 0.5135149 (1025)	total: 55.3s	remaining: 4m 5s
1200:	test: 0.5138190	best: 0.5138190 (1200)	total: 1m	remaining: 4m 3s
1300:	test: 0.5148871	best: 0.5148871 (1300

In [26]:
model.save('model.cbm')

Теперь, имея предикты, можно посчитать метрики качества:

In [24]:
# Predict
y_hat_test = model.predict(X_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Predicted: y_hat_test.shape = (241521,)


In [25]:
def compute_metrics(y_true, y_hat, q):
    for eval_metric in ['NDCG:top=10;type=Exp']:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.512


Ожидаем, что ваша модель покажет результаты выше бейзлайна!